In [23]:
import pandas as pd
import numpy as np
import sqlite3
import requests
from bs4 import BeautifulSoup
from datetime import datetime

In [24]:

url = 'https://web.archive.org/web/20230908091635/https://en.wikipedia.org/wiki/List_of_largest_banks'
table_attribs = ['Name','MC_USD_Billion']
db_name = 'Banks.db'
table_name = 'Largest_banks'
csv_path = 'exchange_rate.csv'
output_path = './Largest_banks_data.csv'

In [25]:

def log_progress(message):
    timestamp_format = '%Y-%h-%d-%H:%M:%S'
    now = datetime.now()
    timestamp = now.strftime(timestamp_format) 
    with open("code_log.txt","a") as f: 
        f.write(timestamp + ' : ' + message + '\n')

log_progress('예비 작업 완료. ETL 프로세스 시작')

In [26]:
def extract(url, table_attribs):
    page=requests.get(url).text
    data = BeautifulSoup(page, 'html.parser')
    df=pd.DataFrame(columns=table_attribs)
    tables = data.find_all('tbody')
    rows = tables[0].find_all('tr')
    for row in rows:
        col = row.find_all('td')
        if len(col)!=0:
            data_dict = {"Name": col[1].text.strip(),
                            "MC_USD_Billion": float(col[2].text.strip().replace("\n",""))}
            df1 = pd.DataFrame([data_dict])
            df = pd.concat([df,df1], ignore_index=True)
    return df

df = extract(url, table_attribs)

log_progress('데이터 추출 완료. 변환 프로세스 시작')

C:\Users\user\AppData\Local\Temp\ipykernel_17668\804448994.py:13: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df,df1], ignore_index=True)


In [27]:
def transform(df, csv_path):
    exchange_df = pd.read_csv(csv_path)
    exchange_rate = dict(zip(exchange_df.iloc[:,0], exchange_df.iloc[:,1]))
    df['MC_GBP_Billion'] = [np.round(x * exchange_rate['GBP'],2) for x in df['MC_USD_Billion']]
    df['MC_EUR_Billion'] = [np.round(x * exchange_rate['EUR'],2) for x in df['MC_USD_Billion']]
    df['MC_INR_Billion'] = [np.round(x * exchange_rate['INR'],2) for x in df['MC_USD_Billion']]
    return df

df = transform(df, csv_path)

log_progress('데이터 변환 완료. 로딩 프로세스 시작')

In [28]:
def load_to_csv(df, output_path):
    df.to_csv(output_path)

load_to_csv(df, output_path)

log_progress('데이터가 CSV 파일에 저장됨')

In [29]:
sql_connection = sqlite3.connect(db_name)

log_progress('SQL 연결 시작됨')

def load_to_db(df, sql_connection, table_name):
    df.to_sql(table_name, sql_connection, if_exists='replace', index=False)
    
load_to_db(df, sql_connection, table_name)

log_progress('데이터가 데이터베이스에 테이블로 로드됨, 쿼리 실행 중')

In [30]:
def run_queries(query_statement, sql_connection):
    print(query_statement)
    query_output = pd.read_sql(query_statement, sql_connection)
    print(query_output)

# query_statement = "SELECT * FROM Largest_banks"
# run_queries(query_statement, sql_connection)

query_statement = "SELECT AVG(MC_GBP_Billion) FROM Largest_banks"
run_queries(query_statement, sql_connection)

# query_statement = "SELECT Name from Largest_banks LIMIT 5"
# run_queries(query_statement, sql_connection)

log_progress('프로세스 완료')

sql_connection.close()

log_progress('서버 연결 종료')

SELECT AVG(MC_GBP_Billion) FROM Largest_banks
   AVG(MC_GBP_Billion)
0              151.987
